<a href="https://colab.research.google.com/github/sabithajaleel/ML-Notebooks/blob/main/ManhattanRecommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install lat-lon-parser

# New Section

In [ ]:
# from bs4 import BeautifulSoup # library to extract data from a website and used for analytics
# import numpy as np # library to handle data in a vectorized manner
# import pandas as pd # library for data analysis
# import requests
# from lat_lon_parser import parse

# touristAttractionsData = pd.DataFrame(columns=["City", "Attractions", "Latitude", "Longitude"])

# r = requests.get("https://en.wikipedia.org/wiki/Tourist_attractions_in_Dubai")

# soup = BeautifulSoup(r.text.replace('\n', ''), "html.parser")

# lis = soup.find("div", {"class": "mw-content-ltr"}).find("ul").find_all("li")

# for li in lis:
#   link = li.find("a", href=True)

#   attraction = link["title"]

#   r = requests.get("https://en.wikipedia.org" + link["href"])
#   soup = BeautifulSoup(r.text.replace('\n', ''), "html.parser")

#   latitudestr = soup.find('span', {"class": "latitude"}).text
#   longitudestr = soup.find('span', {"class": "longitude"}).text
#   latitude = parse(latitudestr)
#   longitude = parse(longitudestr)

#   touristAttractionsData = touristAttractionsData.append({
#       "City": "Dubai",
#       "Attractions": attraction,
#       "Latitude": float(latitude),
#       "Longitude": float(longitude)
#   }, ignore_index=True)

# touristAttractionsData.head()

In [ ]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analysis
import requests
from sklearn.cluster import KMeans # Clustering algorithm
import json # library to handle JSON files

#download dataset
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
#Open the json dataset.
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

#This helps us in knowing which value names to use when iterating to get specific datas
neighborhoods_data = newyork_data['features']

column_names = ['Borough', 'Neighbourhood', 'Latitude', 'Longitude']
neighbourhoods = pd.DataFrame(columns=column_names)

#Pass in data to the new table
for data in neighborhoods_data:
    borough = neighborhoods_name = data['properties']['borough']
    neighborhoods_name = data['properties']['name']

    neighborhoods_latlon = data['geometry']['coordinates']
    neighborhoods_lat = neighborhoods_latlon[1]
    neighborhoods_lon = neighborhoods_latlon[0]
    neighbourhoods = neighbourhoods.append({
        'Borough': borough,
        'Neighbourhood': neighborhoods_name,
        'Latitude': neighborhoods_lat,
        'Longitude': neighborhoods_lon,
    }, ignore_index=True)

# Since we are working with only manhattan data
manhattan = neighbourhoods[neighbourhoods['Borough'] == 'Manhattan'].reset_index(drop=True)

In [12]:
# call Foursquare API for each location and get the nearby venues, within a 1000-meter radius of that location.
import requests

def getNearbyVenues(names, latitudes, longitudes, radius=1000):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        url = "https://api.foursquare.com/v3/places/search"

        latlongarr = [str(lat), str(lng)]
        params = {
            # "query": "Coffee",
  	        "ll": ",".join(latlongarr),
  	        "open_now": "true",
  	        "sort":"DISTANCE"
            }

        headers = {
            "Accept": "application/json",
            "Authorization": "fsq3jJaXevy7ImJxh08qwY/cj25mZLHoVjz5rQWssp/zFPI="
        }

        response = requests.request("GET", url, params=params, headers=headers)
        results = response.json()["results"]
        if results:
          for v in results:
           venues_list.append([(
            name, lat, lng, v['categories'][0]['name'], v['categories'][0]['id'],
           v['geocodes']['main']['latitude'], v['geocodes']['main']['longitude'],
           v['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood      Longitude',  'Venue', 'id', 'Venue Latitude', 'Venue Longitude','Venue Category']

    return(nearby_venues)


In [ ]:
# yasIsland = touristAttractionsData[touristAttractionsData['Attractions'] == 'Yas Island'].reset_index(drop=True)
# attraction_venues = getNearbyVenues(names=yasIsland['Attractions'],
#                                    latitudes=yasIsland['Latitude'],
#                                    longitudes=yasIsland['Longitude']
#                                   )
# attraction_venues.head()

In [13]:
manhattan_venues = getNearbyVenues(names=manhattan['Neighbourhood'],
                                   latitudes=manhattan['Latitude'],
                                   longitudes=manhattan['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [29]:
# One Hot encoding
manhattan_seg_onehot  = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="",prefix_sep="")

#add neighborhood column back to dataframe,  and adjust dataframe columns
manhattan_seg_onehot['Neighbourhood'] = manhattan_venues['Neighbourhood']
fixed_columns = [manhattan_seg_onehot.columns[-1]] + list(manhattan_seg_onehot.columns[:-1])
manhattan_seg_onehot  = manhattan_seg_onehot[fixed_columns]

# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
manhattan_seg_group = manhattan_seg_onehot.groupby('Neighbourhood').mean().reset_index()
# manhattan_seg_group.head()

def return_most_common_values(row, num_top_venues):
  row_categories = row.iloc[1:]
  row_categories_sorted = row_categories.sort_values(ascending=False)

  return row_categories_sorted.index.values[0: num_top_venues]

columns = ['Neighbourhood']
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
manhattan_venues_seg = pd.DataFrame(columns=columns)
manhattan_venues_seg['Neighbourhood'] = manhattan_seg_group['Neighbourhood']
print(manhattan_seg_group.shape[0])

for ind in np.arange(manhattan_seg_group.shape[0]):
    manhattan_venues_seg.iloc[ind, 1:] = return_most_common_values(manhattan_seg_group.iloc[ind, :], num_top_venues)

40


In [32]:
manhattan_venues_seg.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,9/11 Memorial - Pools And Freedom Tower,City Hall Park,"Robert F. Wagner, Jr Park",Liberty Park,La Colombe Torrefaction,9/11 Memorial South Pool,Fearless Girl,iPark - 211 North END Avenue Parking,Battery Park,Atera
1,Carnegie Hill,John Jay Park,Carl Schurz Dog Run,Ladies Pavilion,Conservatory Garden,Bethesda Terrace,Central Park Loop,H&H Bagels,Engineers Gate,Central Park,Starbucks
2,Central Harlem,Riverbank State Park,The Winery,Riverside Park,Conservatory Garden,Central Park,Starbucks,Engineers Gate,Hippo Playground,El Malecon,Absolute Bagels
3,Chelsea,Dough,Little Island,Pier 57 Rooftop Park,Coppelia,Entwine,Fig & Olive,Starbucks,Tesla Store,Pier 63 - Hudson River Park,Hotel Chelsea Lobby Bar
4,Chinatown,Kossar's Bialys,Brooklyn Bridge Park: Main Street,City Hall Park,Sunny and Annie's,Mulberry Iconic Magazines,Bubby's,Brooklyn Bridge,Punjabi Grocery & Deli,Atera,Anthology Film Archives
